In [8]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [9]:
from sailor import RouteGenConfig, SailorDataWarehouse

_config = RouteGenConfig.from_env()
_warehouse = SailorDataWarehouse(_config, db_path="../db/sailor_db.sqlite")

routes = await _warehouse.create_routes(context="flight agency admin panel", count=20)
sessions_group = await _warehouse.create_sessions(routes, count=500)

In [10]:
from typing import List
from sklearn.model_selection import train_test_split
from sailor.types.route_specs import SessionSpec

train_sessions: List[SessionSpec] = []
test_sessions: List[SessionSpec] = []

for sessions in sessions_group:
    _train, _test = train_test_split(sessions, test_size=0.2, random_state=14)
    train_sessions.extend(_train)
    test_sessions.extend(_test)

In [11]:
from sailor.sailor_engine import KNNSailorEngine, SVCSailorEngine

knn_engine = KNNSailorEngine()
knn_engine.fit(routes, train_sessions)

svc_engine = SVCSailorEngine()
svc_engine.fit(routes, train_sessions)

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('svc', LinearSVC(class_weight='balanced', max_iter=2000))])

In [12]:
knn_report = knn_engine.validate(test_sessions)
print("-------------------- KNN -------------------")
print(knn_report)

svc_report = svc_engine.validate(test_sessions)
print("-------------------- SVC -------------------")
print(svc_report)

-------------------- KNN -------------------
                                   precision    recall  f1-score   support

          /admin/settings/general       0.97      0.70      0.81       100
  /admin/bookings/view/:bookingId       0.78      0.94      0.85       100
             /admin/agents/create       0.85      0.93      0.89       100
             /admin/bookings/list       0.79      0.84      0.81       100
                 /admin/dashboard       0.76      0.78      0.77       100
             /admin/reports/sales       0.98      0.94      0.96       100
            /admin/settings/users       0.75      0.85      0.79       100
/admin/customers/view/:customerId       0.80      0.94      0.87       100
            /admin/flights/search       0.85      0.89      0.87       100
         /admin/promotions/create       0.99      0.74      0.85       100
        /admin/notifications/list       0.96      0.82      0.89       100
               /admin/agents/list       0.89      0.82

In [13]:
_query = "Create a new flight"

print("-------------------- KNN -------------------")
routes = knn_engine.predict(_query)[:5]
for route in routes:
  print(f"Path: {route.path}, Score: {route.score:.2f}")
  print(f"Context: {route.context}")

print("-------------------- SVC -------------------")
routes = svc_engine.predict(_query)[:5]
for route in routes:
  print(f"Path: {route.path}, Score: {route.score:.2f}")
  print(f"Context: {route.context}")

-------------------- KNN -------------------
Path: /admin/flights/create, Score: 0.28
Context: admin flights create flights create new flight admin panel flight agency Devise new flight Searched destination airport New flight creation new flights Admin panel create new flight Navigated create flight page schedule new cargo flight Creating new flight hotel booking option Tried create flight invalid flight number format Adding new flight specific meal options Adding flight entertainment details flight information Setting arrival airport Singapore Changi create fligh Creating new flight specific destination build new journey Creating new flight connecting train option New flight configuration book new plane ticket Creating new flight specific aircraft type Setting aircraft type Boeing Freighter Navigated Manage Flights section Setting departure airport Dubai International Creating new flight flexible booking options create flight Setting aircraft type Boeing Dreamliner create new flight N

In [ ]:
_models_path = "../build/models"
knn_engine.save_model("knn_model", _models_path)
svc_engine.save_model("svc_model", _models_path)